In [1]:
from configparser import ConfigParser
import psycopg2
import numpy as np

In [4]:
def config(filename='../psql_sample.ini', section='postgresql'):
    parser = ConfigParser()
    # read config file
    parser.read(filename)
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
    
    return db

In [5]:
cfg = config()

In [6]:
#Establish the connection and create a cursor to the database
try:
    print("Here's an attempt to connect to the database")
    conn = psycopg2.connect(**cfg)
    cursor = conn.cursor()
    print("Look's like it was a success")
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Here's an attempt to connect to the database
Look's like it was a success


In [7]:
dataframe_query = '''
select 
	neighborhood,
	count(*) as theft_from_vehicle_counts
from t_fact_table
left join t_crime_dim on t_crime_dim.crime_key = t_fact_table.crime_key
left join t_location_dim on t_location_dim.location_key = t_fact_table.location_key
where crime_type = 'Theft from Vehicle'
group by neighborhood
order by theft_from_vehicle_counts desc
'''
try:
    #Run a simple query
    cursor.execute(dataframe_query)
    #Loop over the result set and print first 10 rows
    result_list = cursor.fetchall()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [8]:
#close connection
cursor.close()
conn.close()

In [10]:
import pandas as pd
data_set = pd.DataFrame(result_list, columns= ['Neighborhood', 'theft_from_vehicle_counts'])
data_set.head()

,Neighborhood,theft_from_vehicle_counts
0,Central Business District,23777
1,West End,6680
2,Mount Pleasant,4121
3,Strathcona,3439
4,Fairview,3427


In [11]:
data_set.to_csv('./clustering_dataset.csv', index=False)

In [14]:
print(data_set.Neighborhood)

0      Central Business District
1                       West End
2                 Mount Pleasant
3                     Strathcona
4                       Fairview
                 ...            
98                    sun-valley
99                       skyland
100                    wellshire
101                 indian-creek
102                     Musqueam
Name: Neighborhood, Length: 103, dtype: object
